In [ ]:
import numpy as np
import sympy
import qsymm

sympy.init_printing(print_builtin=True)
np.set_printoptions(precision=2, suppress=True, linewidth=150)

# Wire with chiral symmetry

Kwant is used to generate the tight-binding model for a finite cross section wire

In [ ]:
import kwant
import matplotlib.pyplot as plt
import wire.funcs as funcs

Define symmetry candidates, in 1D the only nontrivial space-group element is inversion, besides add both TR and PH.

The $\mathbf{k\cdot p}$ Hamiltonian is discretized with a lattice constant of $a = 10$ nm. The wire and shell diameter of $r_1 = 15$ nm, $r_2 = 30$ nm used here allow the calculation in about 30 seconds, the example shown in the paper uses $r_1 = 25$ nm, $r_2 = 40$ nm and takes about 10 minutes.

The electric field is in the $z$ direction all along.

In [ ]:
candidates = {qsymm.PointGroupElement(np.eye(1), True, False, None),
              qsymm.PointGroupElement(np.eye(1), False, True, None),
              qsymm.PointGroupElement(-np.eye(1), False, False, None)
             }
candidates = qsymm.groups.generate_group(candidates)
len(candidates)

r1=15
r2=30
Vz = lambda *pos: pos[2]
color = lambda site: ('w' if (np.dot(site.pos, site.pos)) < r1**2 else 'r')

Make system with mirror symmetry and magnetic field in $x$ direction.

In [ ]:
syst_pars = dict(a=10, angle=0, coverage_angle=100, r1=r1, r2=r2, shape='circle',
                 A_correction=True, with_shell=True, rotate_spin_orbit=False)

params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

lead = funcs.make_lead(**syst_pars)

kwant.plot(lead, site_color = color);

In [ ]:
leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The system has the full 8-element symmetry group.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Add magnetic field in $z$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0.1, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e**(I * k)'): V,
                   sympy.sympify('e**(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The symmetry group still contains an effective time-reversal symmetry $M_y T$

In [ ]:
%time symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Add magnetic field in $y$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0.1, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The symmetry group only contains $C_{2z} T$, which doesn't prevent band-tilting.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Move shell such that it breaks $M_y$

In [ ]:
syst_pars = dict(a=10, angle=45, coverage_angle=100, r1=r1, r2=r2, shape='circle',
                 A_correction=True, with_shell=True, rotate_spin_orbit=False)

lead = funcs.make_lead(**syst_pars)

kwant.plot(lead, site_color = color);

Field only along the wire.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

Mirror symmetry $M_x$ is preserved.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Add field in $z$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0.1, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.Model({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

All symmetry is broken except for particle-hole.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

## Using kwant integration

In [ ]:
candidates = {qsymm.PointGroupElement(np.eye(1), True, False, None),
              qsymm.PointGroupElement(np.eye(1), False, True, None),
              qsymm.PointGroupElement(-np.eye(1), False, False, None)
             }
candidates = qsymm.groups.generate_group(candidates)
len(candidates)

r1=15
r2=30
Vz = lambda *pos: pos[2]
color = lambda site: ('w' if (np.dot(site.pos, site.pos)) < r1**2 else 'r')

Make system with mirror symmetry and magnetic field in $x$ direction.

In [ ]:
syst_pars = dict(a=10, angle=0, coverage_angle=100, r1=r1, r2=r2, shape='circle',
                 A_correction=True, with_shell=True, rotate_spin_orbit=False)

params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

lead = funcs.make_lead(**syst_pars)

kwant.plot(lead, site_color = color);

In [ ]:
leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.BlochModel({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The system has the full 8-element symmetry group.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

In [ ]:
MyT = list(symset)[4]

In [ ]:
MyT.conjugate, MyT.antisymmetry

In [ ]:
symset0 = symset

Add magnetic field in $z$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0.1, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.BlochModel({sympy.sympify(1): H0,
                   sympy.sympify('e**(I * k)'): V,
                   sympy.sympify('e**(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The symmetry group still contains an effective time-reversal symmetry $M_y T$

### Missing!!! Check that everything works correctly

In [ ]:
%time symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

In [ ]:
MyT.apply(model) - model

In [ ]:
from qsymm.model import BlochCoeff
import scipy.linalg as la

In [ ]:
[la.norm((g.apply(model) - model)[BlochCoeff(np.array([0]), sympy.sympify(1))]) for g in symset0]

In [ ]:
[la.norm((g.apply(model) - model)[BlochCoeff(np.array([1]), sympy.sympify(1))]) for g in symset0]

In [ ]:
[la.norm((g.apply(model) - model)[BlochCoeff(np.array([-1]), sympy.sympify(1))]) for g in symset0]

In [ ]:
[g.apply(model) == model for g in symset0]

Add magnetic field in $y$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0.1, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.BlochModel({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

The symmetry group only contains $C_{2z} T$, which doesn't prevent band-tilting.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Move shell such that it breaks $M_y$

In [ ]:
syst_pars = dict(a=10, angle=45, coverage_angle=100, r1=r1, r2=r2, shape='circle',
                 A_correction=True, with_shell=True, rotate_spin_orbit=False)

lead = funcs.make_lead(**syst_pars)

kwant.plot(lead, site_color = color);

Field only along the wire.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.BlochModel({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

Mirror symmetry $M_x$ is preserved.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]

Add field in $z$ direction.

In [ ]:
params = dict(alpha=20, B_x=0.5, B_y=0, B_z=0.1, Delta=110, g=50,
              orbital=True, mu_sc=100, mu_lead=10, c_tunnel=3/4,
              V=Vz,  **funcs.constants.__dict__)

leadf = lead.finalized()
H0 = leadf.cell_hamiltonian(params=params)
V = leadf.inter_cell_hopping(params=params)
print(H0.shape, V.shape)
model = qsymm.BlochModel({sympy.sympify(1): H0,
                   sympy.sympify('e^(I * k)'): V,
                   sympy.sympify('e^(-I * k)'): V.T.conj()},
                  momenta=[sympy.sympify('k')])

All symmetry is broken except for particle-hole.

In [ ]:
symset, cg = qsymm.symmetries(model, candidates, sparse_linalg=True)
print(len(symset), len(cg))
[(s.R, s.conjugate, s.antisymmetry) for s in symset]